# Latitude, Longitude for any pixel in a GeoTiff File
How to generate the latitude and longitude for a pixel at any given position in a GeoTiff file.

In [1]:
import sys
try:
    from osgeo import ogr, osr, gdal
except:
    sys.exit('ERROR: cannot find GDAL/OGR modules')

In [2]:
ds = gdal.Open('/Users/macbook/Documents/BTP/Satellite/Data/LC08_L1TP_150041_20171027_20171027_01_RT/LC08_L1TP_150041_20171027_20171027_01_RT_B2.TIF')

In [3]:
col, row, band = ds.RasterXSize, ds.RasterYSize, ds.RasterCount
print(col, row, band)

(7551, 7701, 1)


In [4]:
xoff, a, b, yoff, d, e = ds.GetGeoTransform()
print(xoff, a, b, yoff, d, e)


# details about the params:
# xoff = 
# a = 
# b = 

(619185.0, 30.0, 0.0, 3151515.0, 0.0, -30.0)


In [5]:
def pixel2coord(x, y):
    """Returns global coordinates from pixel x, y coords"""
    xp = a * x + b * y + xoff
    yp = d * x + e * y + yoff
    return(xp, yp)

In [6]:
x,y = pixel2coord(3500,3500)
print (x, y)

(724185.0, 3046515.0)


In [7]:
print pixel2coord(12,6)

(619545.0, 3151335.0)


In [8]:
# get the existing coordinate system
# ds = gdal.Open('path/to/file')
old_cs= osr.SpatialReference()
old_cs.ImportFromWkt(ds.GetProjectionRef())

# create the new coordinate system
wgs84_wkt = """
GEOGCS["WGS 84",
    DATUM["WGS_1984",
        SPHEROID["WGS 84",6378137,298.257223563,
            AUTHORITY["EPSG","7030"]],
        AUTHORITY["EPSG","6326"]],
    PRIMEM["Greenwich",0,
        AUTHORITY["EPSG","8901"]],
    UNIT["degree",0.01745329251994328,
        AUTHORITY["EPSG","9122"]],
    AUTHORITY["EPSG","4326"]]"""
new_cs = osr.SpatialReference()
new_cs.ImportFromWkt(wgs84_wkt)

# create a transform object to convert between coordinate systems
transform = osr.CoordinateTransformation(old_cs,new_cs) 

In [9]:
latlong = transform.TransformPoint(x,y)

In [10]:
print (latlong)

(71.26985566679045, 27.52388207534747, 0.0)


In [11]:
# Keep this small Notebook to give example how to extract lat, lon, for one pixel, of a geotiff
# with all the explanation from the stack overflow example.
# 
# Make separate notebook to get latlong for all the pixels sytematically, using the file name tricks in python 
# (probably from DL Andrew NG Course)